In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, Embedding, Flatten, Dense, Concatenate, Dot
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
df = pd.read_csv('./ratings.csv')

In [ ]:
N = len(df['user_id'].unique()) + 1
M = len(df['book_id'].unique()) + 1

In [ ]:
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [ ]:
K = 10 
mu = df_train.rating.mean()
epochs = 15
reg = 0.

u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K)(u) 
m_embedding = Embedding(M, K)(m) 
u_embedding = Flatten()(u_embedding)
m_embedding = Flatten()(m_embedding)
x = Concatenate()([u_embedding, m_embedding])

x = Dense(400)(x)
x = Activation('relu')(x)
x = Dense(1)(x)

model = Model(inputs=[u, m], outputs=x)

In [ ]:
model.compile(
  loss='mse',
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['mse'],
)

In [ ]:
r = model.fit(
  x=[df_train.user_id.values, df_train.book_id.values],
  y=df_train.rating.values - mu,
  epochs=epochs,
  batch_size=128
)